In [8]:
import pandas as pd


def parse_qna_sheet():
    filepath = "xlsx/QnA Informasi Data Umum ITB.xlsx"

    df = pd.read_excel(filepath, sheet_name="Sheet1")
    df = df.dropna()
    df["Pertanyaan"] = None
    df["Jawaban"] = None

    for i, row in df.iterrows():
        try:
            line = row["QnA"]
            jawaban_token = ""
            if "Pertanyaan: " in line:
                line = line.split("Pertanyaan: ")[1]
                jawaban_token = "Jawaban: "
            elif "Question: " in line:
                line = line.split("Question: ")[1]
                jawaban_token = "Answer: "

            pertanyaan, jawaban = line.split(jawaban_token)
            df.at[i, "Pertanyaan"] = pertanyaan
            df.at[i, "Jawaban"] = jawaban
        except Exception as e:
            print(e)
            print(i)
            print(jawaban_token)
            print(row["QnA"])

    df.to_excel("xlsx/QnA Informasi Data Umum ITB.xlsx", index=False)


filepath = "xlsx/QnA Informasi Data Umum ITB.xlsx"
df = pd.read_excel(filepath, sheet_name="Sheet1")
df["LLM Response"] = None
df["Docs"] = None

In [10]:
from langserve import RemoteRunnable

chain = RemoteRunnable("http://localhost:6900/chatbot")

filepath = "xlsx/Jawaban_Llama3_8B_New.csv"
# df = pd.read_csv(filepath)

for i, row in df.iterrows():

    if not pd.isnull(row["LLM Response"]):
        continue

    prompt = row["Pertanyaan"]
    print(f"Asking question {str(i + 1)}: {prompt}")
    response = chain.invoke(
        {
            "input": prompt,
        },
        config={
            "configurable": {"user_id": f"owen_llama38b_new_{i}"},
        },
    )
    answer = response["answer"]
    print(f"Answer: {answer}")
    df.at[i, "LLM Response"] = response["answer"]
    df.at[i, "Docs"] = response["context"]
    df.to_csv(filepath, index=False)

    print()

df

Asking question 363: Berapa daya tampung SNBP untuk program studi FMIPA-IPA pada tahun 2024? 

Answer: Berdasarkan dokumen yang diberikan, daya tampung SNBP untuk program studi FMIPA-IPA pada tahun 2024 adalah sebagai berikut:

* Fisika: 20
* Astronomi: 9
* Kimia: 17

Jadi, total daya tampung SNBP untuk program studi FMIPA-IPA adalah 46.

Sumber: Dokumen yang diberikan, referensi: https://admission.itb.ac.id/info/info-pendaftaran-program-sarjana/

Asking question 364: Program studi mana yang memiliki keketatan kurang dari 5% pada tahun 2024? 

Answer: Berdasarkan dokumen yang diberikan, program studi yang memiliki keketatan kurang dari 5% pada tahun 2024 adalah:

* Kode: 332021 Fakultas/Sekolah: SEKOLAH TEKNIK ELEKTRO DAN INFORMATIKA – KOMPUTASI (STEI-K) Keketatan: < 5%

Referensi: Dokumen Kode 332021.

Asking question 365: Berapa daya tampung total untuk jalur SM-ITB di Fakultas Teknik Mesin dan Dirgantara (FTMD) pada tahun 2024? 

Answer: Berdasarkan dokumen yang diberikan, tidak ada

,No.,QnA,Topik,Link Sumber Dokumen,Pertanyaan,Jawaban,LLM Response,Docs
0,1,Pertanyaan: Berapa fakultas/sekolah yang dimil...,Penerimaan - Sarjana,https://docs.google.com/spreadsheets/d/1T4z6Bw...,Berapa fakultas/sekolah yang dimiliki oleh ITB...,ITB memiliki dua belas fakultas/sekolah dalam ...,"Berdasarkan dokumen, ITB memiliki 12 fakultas/...",[page_content='['50 Program Studi Sarjana' 'In...
1,2,Pertanyaan: Apa yang harus dilakukan mahasiswa...,Penerimaan - Sarjana,https://docs.google.com/spreadsheets/d/1T4z6Bw...,Apa yang harus dilakukan mahasiswa baru ITB se...,Mahasiswa baru ITB harus diterima menjadi maha...,"Menurut informasi yang diperoleh, sebelum memi...",[page_content='program peminatan. Pemilihan pr...
2,3,Pertanyaan: Berdasarkan apa penempatan mahasis...,Penerimaan - Sarjana,https://docs.google.com/spreadsheets/d/1T4z6Bw...,Berdasarkan apa penempatan mahasiswa di salah ...,Penempatan mahasiswa di salah satu program stu...,"Berdasarkan dokumen, penempatan mahasiswa di s...",[page_content='['Institut Teknologi Bandung (I...
3,4,Pertanyaan: Apa nama fase di mana mahasiswa ba...,Penerimaan - Sarjana,https://docs.google.com/spreadsheets/d/1T4z6Bw...,Apa nama fase di mana mahasiswa baru ITB diter...,Fase tersebut disebut sebagai Tahap Persiapan ...,"Menurut dokumen, fase di mana mahasiswa baru I...","[page_content='['ITB, sebagai perguruan tinggi..."
4,5,Pertanyaan: Di mana saja lokasi kampus ITB yan...,Penerimaan - Sarjana,https://docs.google.com/spreadsheets/d/1T4z6Bw...,Di mana saja lokasi kampus ITB yang digunakan ...,Lokasi kampus ITB yang digunakan untuk menunja...,Lokasi kampus ITB yang digunakan untuk menunja...,"[page_content='['ITB, sebagai perguruan tinggi..."
...,...,...,...,...,...,...,...,...
876,877,Pertanyaan: Apa itu komunitas Teknik Fisika? \...,Pertanyaan Umum Tentang Teknik Fisika ITB,https://tf.itb.ac.id/pertanyaan-umum/,Apa itu komunitas Teknik Fisika? \n,"Seiring perkembangan keilmuan, program-program...",Komunitas Teknik Fisika adalah sebuah kelompok...,[page_content='['Apa itu Teknik Fisika?' 'Tekn...
877,878,Pertanyaan: Di manakah biasanya lulusan Teknik...,Pertanyaan Umum Tentang Teknik Fisika ITB,https://tf.itb.ac.id/pertanyaan-umum/,Di manakah biasanya lulusan Teknik Fisika ITB ...,"Dengan dasar ilmu yang luas, lulusan TF memili...",Lulusan Teknik Fisika ITB memiliki kesempatan ...,[page_content='(S.T.) dan Magister Teknik (M.T...
878,879,"Pertanyaan: Saya bukan lulusan Teknik Fisika, ...",Pertanyaan Umum Tentang Teknik Fisika ITB,https://tf.itb.ac.id/pertanyaan-umum/,"Saya bukan lulusan Teknik Fisika, bisakah saya...",Bergantung dengan latar pendidikan yang anda m...,Bergantung dengan latar pendidikan yang Anda m...,[page_content='(S.T.) dan Magister Teknik (M.T...
879,880,Pertanyaan: Apakah ada sub-jurusan di Teknik F...,Pertanyaan Umum Tentang Teknik Fisika ITB,https://tf.itb.ac.id/pertanyaan-umum/,Apakah ada sub-jurusan di Teknik Fisika ITB? \n,Lulusan dari prodi S1 / S2 Teknik Fisika atau ...,Menurut informasi yang saya dapat dari dokumen...,[page_content='['Apa itu Teknik Fisika?' 'Tekn...


In [26]:
from evaluate import load

# the en text is from index 525 to 693
en_start_idx = 525
en_end_idx = 693

prediction_en = df["Jawaban"].iloc[en_start_idx:en_end_idx]
reference_en = df["LLM Response"].iloc[en_start_idx:en_end_idx]

# id text is all the other text
prediction_id = df["Jawaban"].iloc[:en_start_idx] + df["Jawaban"].iloc[en_end_idx:]
reference_id = (
    df["LLM Response"].iloc[:en_start_idx] + df["LLM Response"].iloc[en_end_idx:]
)


bertscore = load("bertscore")

results_en = bertscore.compute(
    predictions=prediction_en.tolist(), references=reference_en.tolist(), lang="en"
)

results_id = bertscore.compute(
    predictions=prediction_id.tolist(), references=reference_id.tolist(), lang="id"
)
# results

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 936.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 233.38 MiB is free. Process 1807 has 80.00 MiB memory in use. Including non-PyTorch memory, this process has 5.42 GiB memory in use. Of the allocated memory 4.97 GiB is allocated by PyTorch, and 356.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)